In [1]:
from IPython.display import display, HTML
import refinitiv.dataplatform as rdp
import configparser as cp
import json
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout
from datetime import datetime, timedelta


config = cp.ConfigParser()
config.read("c:/Refinitiv/config.cfg")
#session = rdp.DesktopSession(app_key=config['session']['app_key'])
session = rdp.PlatformSession(
    config['session']['app_key'],
    rdp.GrantPassword( username=config['session']['user'], password=config['session']['password'] )
)
session.open()

endpoint = rdp.Endpoint(session, 'https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts')


#### Define a Swap request to provide an Accrued Percentage value replicating the NatWest Reference Rate
Input values
* Start date of the IR Swap matching the start date of the realized rate period  
* Maturity date of the IR Swap matching the end date of the realized rate period  
* Amount used to compute the interest paid over the realized rate period
* Currency of the amount  
* Spread in Basis point that is added to the realized rate  
* Day count basis used to compute each daily year fraction  
* Name of the reference index used to compute the realized rate (e.g. SONIA, SOFR)  
* Look back period i.e. number of days between the daily interest period and the reference index publication

In [2]:
import dateutil.parser
from datetime import date

def create_swap(start_date, end_date, amount, spread_bp, day_count, currency, index_name, look_back):
    
    swap_definition = {
        "instrumentType":"Swap", 
        "instrumentDefinition": {
            "instrumentTag":"Compounding-Rate",
            #"template": "OIS_SONIA",
            "startDate":start_date,
            "tenor":"1Y",
            "legs":[
            {
                "legTag": "Fixed",
                "interestType": "Fixed",
                "notionalCcy": currency,
                "notionalAmount": amount,
                "interestCalculationMethod": day_count,
                "direction":"Paid",
                "interestType":"Fixed",
                "interestPaymentFrequency": "Zero",
                "fixedRatePercent":1.0,
            },
            {
                "legTag": "Float",
                "direction":"Received",
                "interestType":"Float",
                "notionalCcy": currency,
                "notionalAmount": amount,
                "spreadBp": spread_bp,
                "interestPaymentFrequency": "Zero",
                "interestCalculationMethod": day_count,
                #"indexFixingRIC": index_name,
                "indexTenor": "ON",
                "indexName": index_name,
                "indexFixingLag": look_back,
                "indexCompoundingMethod": "Compounded",
            }]
        },
        
        "pricingParameters": {
            "valuationDate": end_date,
        },
    }
    
    return swap_definition


#### Define the IR Swap using the above function & submit to the Financial Contract API endpoint
And then display the response as a user friendly dataframe

In [3]:
def swap_accrued_percent(start_date, end_date, amount, spread_bp, day_count, currency, index_name, look_back):
    request_body = {

        "fields" : ["InstrumentTag","LegTag","MarketValueInDealCcy","AccruedPercent", "AccruedAmountInDealCcy",\
                    "ErrorCode","ErrorMessage"],

        "universe" : [
            create_swap(start_date, end_date, amount, spread_bp, day_count, currency, index_name, look_back)
        ],

        "outputs" : ["Data","Headers"],

    }

    swap = endpoint.send_request(
        method = rdp.Endpoint.RequestMethod.POST,
        body_parameters = request_body)

    print("IR Swap RESPONSE:")
    ## print(json.dumps(swap.data.raw, indent=2))
    
    headers_name = [h['name'] for h in swap.data.raw['headers']]
    df = pd.DataFrame(data=swap.data.raw['data'], columns=headers_name)
    display(df)
    
    # Floating rate Accrued Percentage
    return df['AccruedPercent'][1]


#### Function to compute the year fraction of the realized rate period.
- Start date & End Date matching the IR Swap definition
- Day Count Basis as per the IR Swap
- Period Type set to Year to get the year fraction of the period
- For Calendars I am just re-using the currency input of the IR Swap

Submit the Request to the relevant Quantitative Analytics endpoint & then display the response in a reader friendly format


In [4]:
def rate_period(start_date, end_date, day_count_basis, period_type, calendars):
    
    cp_endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics-dates-and-calendars/beta1/count-periods")

    cp_request_body=[
        {
        "tag": "my request 1",
        "startDate": start_date,
        "endDate": end_date,
        "DayCountBasis":day_count_basis,
        "periodType": period_type,
        "calendars": [calendars]
        }
    ]


    period_of_time = cp_endpoint.send_request(
        method = rdp.Endpoint.RequestMethod.POST,
        body_parameters = cp_request_body)

    print("\nCount Periods RESPONSE:")
    #print(json.dumps(period_of_time.data.raw, indent=2))
    
    df = pd.json_normalize(period_of_time.data.raw)
    display(df)
    
    return period_of_time.data.raw[0]['count']



#### Finally the Swap Accrued is divided by the Period Year Fraction to give us the Annualized RFR for the period considered.

In [5]:
def calc_realized_rate(start_date, end_date, amount, spread_bp, day_count, currency, index_name, look_back, dcb, period_type):
    
    accrued_percent = swap_accrued_percent(start_date, end_date, amount, spread_bp, day_count, currency, index_name, look_back)
    period_of_time = rate_period(start_date, end_date, dcb,period_type, currency)
    
    return round(accrued_percent / period_of_time,4)
    

#### Read in our yser supplied values from the UI, calculate the rate and then update the UI

In [6]:
def calculate_rr(value):
    
    rrf_rate_output.value = '...'
    
    indices = {
        'GBP' : 'SONIA',
        'EUR' : 'ESTER',
        'USD' : 'SOFR'
        }

    start_date = start_date_picker.value.strftime('%Y-%m-%d')
    end_date = end_date_picker.value.strftime('%Y-%m-%d')
    amount = float(amount_input.value)
    spread_bp = float(spb_input.value)*100
    day_count = day_cnt_dd.value
    currency = ccy_dd.value
    look_back = lags_dd.value.split('d')[0]
    index_name = indices[currency]
    
    realized_rf_rate = calc_realized_rate(start_date, end_date, amount, spread_bp, day_count, currency, index_name, look_back, 'A5', 'Year')

    rrf_rate_output.value = str(realized_rf_rate)
    
    return

#### The ipywidgets UI setup code
Define the  
* Calculate Button
* Start date + Maturity date pickers, 
* Currency, DCB + lookback dropdowns 
* Amount and SPBinput fields 
* Realised RFR output field

In [7]:
calculate_btn = widgets.Button(description='Calculate', layout={'width':'90px','height':'40px'})
calculate_btn.on_click(calculate_rr)

startdate = datetime.strptime('2020-02-24', '%Y-%m-%d')
start_date_picker = widgets.DatePicker(description='Start date:', value = startdate,
                                       continuous_update=False, layout={'width':'230px'})
enddate = datetime.strptime('2020-03-24', '%Y-%m-%d')
end_date_picker = widgets.DatePicker(description='End date:', value = enddate,
                                     continuous_update=False, layout={'width':'230px'})
amount_input = widgets.FloatText(description='Amount £:', value=10000.0, 
                                   continuous_update=False, layout={'width':'150px'})
spb_input = widgets.FloatText(description='SPB %:', value=0.0, 
                                 continuous_update=False, layout={'width':'130px'})
currencies=['GBP','EUR','USD']
ccy_dd = widgets.Dropdown(description='Currency:', options=currencies, value='GBP', 
                                 continuous_update=False, layout={'width':'150px'})
day_cnts=['Dcb_Actual_365','Dcb_Actual_360','Dcb_30_360_US']
day_cnt_dd = widgets.Dropdown(description='Day Count:', options=day_cnts, value='Dcb_Actual_365', 
                                 continuous_update=False, layout={'width':'220px'})

lags = [str(i)+'d' for i in range(15)]
lags_dd = widgets.Dropdown(description='Lookback:', options=lags, value='0d', 
                                 continuous_update=False, layout={'width':'140px'})

style = {'description_width': 'initial'}
rrf_rate_output = widgets.Text(value='...',description='Realized RFR %: ', 
                            disabled=True, layout={'width':'160px', 'height':'50px'}, style=style )


inputs_ui_top = widgets.HBox([
    start_date_picker, end_date_picker, 
    amount_input, spb_input], 
    layout={'height':'50px'})

inputs_ui_lower = widgets.HBox([
    ccy_dd, day_cnt_dd,
    lags_dd], 
    layout={'height':'50px'})


outputs_ui = widgets.HBox([calculate_btn,rrf_rate_output],layout={'height':'70px'})

rates_ui = widgets.VBox([inputs_ui_top, inputs_ui_lower, outputs_ui])

accordion = widgets.Accordion(children=[rates_ui], layout={'width':'1000px','height':'330px'})
accordion.set_title(0, 'RATE CALCULATOR')

realised_rfr_calc = widgets.VBox([accordion])
realised_rfr_calc


#### Display the reponse data we get back from the API requests

In [8]:
#calculate_rr('')